In [1]:
!pip install -q langchain pypdf sentence-transformers chromadb openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 라이브러리
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# pdf 로드
loaders = [
    PyPDFLoader("/content/drive/MyDrive/온디바이스AI.pdf"),
    PyPDFLoader("/content/drive/MyDrive/갤럭시 S24 메뉴얼.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load_and_split())

In [5]:
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# 벡터DB
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=ko_embedding
)
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter
)

In [7]:
retriever.add_documents(docs, ids=None)

In [8]:
sub_docs = vectorstore.similarity_search("갤럭시 S24")

In [9]:
print("글 길이: {}\n\n".format(len(sub_docs[0].page_content)))
print(sub_docs[0].page_content)

글 길이: 487


빅스비 비전
86
 멀티윈도우(여러 앱 동시
에 사용하기)
89
 삼성 인터넷
90
 Samsung P
ay(삼성 페이)차례
93 삼성 헬스
9
4
 Samsung Not
es(삼성 노트)
98
 Samsung Members(삼
성 멤버스)
99
 Samsung Kids(삼성 키
즈)
99
 Samsung Gl
obal Goals(삼성 글로벌 골)
99
 Samsung Find(삼성 파
인드)
100
 삼성닷컴
100
 Galax
y Wearable(갤럭시 웨어러블)
100
 PENUP(펜업)(
Galaxy S24 Ultra)
100
 캘린더
101
 리마인더(할 일 알림 받기
)
102
 라디오
103
 음성 녹음
104
 내 파일(파일 확인 및 관리
하기)
104
 시계
105
 계산기
105
 Gaming Hub(게이밍 허
브)
106
 게임 부스터(게임 환경 설
정하기)
107
 Smar
tThings(스마트싱스)
108
 콘텐츠(파일) 공유하기
109


In [10]:
retrieved_docs = retriever.get_relevant_documents("갤럭시 S24")

In [11]:
print("글 길이: {}\n\n".format(len(retrieved_docs[0].page_content)))
print(retrieved_docs[0].page_content)

글 길이: 937


2
시작하기
4 각 부분의 이름과 역할
11
 배터리 충전하기
15
 Nano-SIM 카드 및 eSIM
19
 전원 켜기/끄기
20
 제품 초기 설정
20
 삼성 계정
20
 이전 기기의 데이터 가져
오기(Smart Switch)
22
 화면 알아두기
30
 알림창
31
 화면 캡처 및 화면 녹화
33
 문자 입력
35
 텍스트 추출
앱과 기능 알아보기
36 앱 설치 및 관리
37
 S펜(
Galaxy S24 Ultra)
50
 전화
54
 연락처
56
 메시지
58
 카메라
7
4
 갤러리
7
9
 AR 존
84
 빅스비
85
 빅스비 비전
86
 멀티윈도우(여러 앱 동시
에 사용하기)
89
 삼성 인터넷
90
 Samsung P
ay(삼성 페이)차례
93 삼성 헬스
9
4
 Samsung Not
es(삼성 노트)
98
 Samsung Members(삼
성 멤버스)
99
 Samsung Kids(삼성 키
즈)
99
 Samsung Gl
obal Goals(삼성 글로벌 골)
99
 Samsung Find(삼성 파
인드)
100
 삼성닷컴
100
 Galax
y Wearable(갤럭시 웨어러블)
100
 PENUP(펜업)(
Galaxy S24 Ultra)
100
 캘린더
101
 리마인더(할 일 알림 받기
)
102
 라디오
103
 음성 녹음
104
 내 파일(파일 확인 및 관리
하기)
104
 시계
105
 계산기
105
 Gaming Hub(게이밍 허
브)
106
 게임 부스터(게임 환경 설
정하기)
107
 Smar
tThings(스마트싱스)
108
 콘텐츠(파일) 공유하기
109
 Music Shar
e(뮤직 쉐어)
110
 Smar
t View(스마트 뷰)(TV 화면으로 보기)
111
 Windo
ws와 연결(컴퓨터와 연결해 사용하기)
112
 Samsung D
eX(삼성 덱스)
116
 Googl
e(구글) 앱
117
 이동통신 사업자 앱
